In [1]:
import re
import pandas as pd
import numpy as np
import requests
from lxml import html, etree

## https://news.bitcoin.com/


In [ ]:
## http://docs.python-guide.org/en/latest/scenarios/scrape/
##

class news_bitcoin_parser():
    def __init__(self):
        self.link = 'https://news.bitcoin.com/'
        self.save_dir = "news_bitcoin/"
    
    def read_article_text(self,article_href):
        article_page = requests.get(article_href)
        article_html = html.fromstring(article_page.content)
        article_content = article_html.xpath('//div[@class="td-post-content"]')
        text = etree.tostring(article_content[0])
        text = text.decode("utf-8")  
        return text
    
    def clean_text (self,article_text):
        article_text = re.sub("<.*?>","",article_text)
        article_text = re.sub("&#[0-9]+;|\n"," ",article_text)
        article_text = re.sub(" {1,}"," ",article_text)

        return article_text

    def parse_page(self,page_num, verbose = True):
        page_link = self.link
        if page_num>1:
            page_link = self.link + "/page/" + str(page_num)
        if page_num<=0:
            return 0
        page = requests.get(page_link)
        page_html = html.fromstring(page.content)
        
        news = page_html.xpath('//div[@class="td-container td-pb-article-list"]')
        articles = news[0].xpath('.//div[@class="item-details"]')

        ## for debugging
        self.page = page
        self.page_html = page_html
        self.articles = articles 
        if verbose: print ('read page ' + str(page_num))
        
        for i_article in articles:
            article_name = i_article.xpath(".//a/@title")[0]
            article_time = i_article.xpath(".//time/@datetime")[0]
            article_date = article_time[0:10]
            article_href = i_article.xpath(".//a/@href")[0]
            article_text = self.read_article_text(article_href)
            
            ## cleaning article from special symbols
            article_text = self.clean_text(article_text)
                        
            ## saving article
            fn = re.sub("[^a-zA-Z0-9 ]","",article_name[0:20])
            file_name = self.save_dir + article_date + "_" + 'p_' + '{num:03d}'.format(num=page_num) + "_" + fn + ".txt"
            if verbose: print (file_name)
            f = open(file_name,'w', encoding = 'utf-8')
            f.write(article_name + "\n")
            f.write(article_time+ "\n")
            f.write(article_text)
            f.close()
        print ("page " + str(page_num) + " parsed")

            
bc_parser = news_bitcoin_parser()

for i in range(1,619):
    bc_parser.parse_page(i,verbose=False)

page 1 parsed
page 2 parsed
page 3 parsed
page 4 parsed
page 5 parsed
page 6 parsed
page 7 parsed
page 8 parsed
page 9 parsed
page 10 parsed
page 11 parsed
page 12 parsed
page 13 parsed
page 14 parsed
page 15 parsed
page 16 parsed
page 17 parsed
page 18 parsed
page 19 parsed
page 20 parsed
page 21 parsed
